## Overfitting and validation homework

In this homework, we will cover how to implement the regularization techniques we learned in class:
- Early Stopping
- Weight Decay (L2 Regularization)
- Dropout
- Data Augmentation

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist, cifar10
from keras import regularizers
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)

Using TensorFlow backend.


In [2]:
def create_model():
    """ Creates a new convolutional neural network and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

### Image preprocessing

This code block loads our dataset and does some initial preprocessing. We are using the `MNIST` dataset of hand written digits. You can read about MNIST here: http://yann.lecun.com/exdb/mnist/

In [3]:
(x_train, y_train), (x_val, y_val) = cifar10.load_data()
batch_size = 128
num_classes = 10
epochs = 150

# input image dimensions
img_rows, img_cols = 32, 32

num_train = 1000
num_val = 500  # We choose such a large validation set to illustrate the effects of overfitting without noise

x_train = x_train[:num_train]
y_train = y_train[:num_train]
x_val = x_val[:num_val]
y_val = y_val[:num_val]

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

# This function will print the training
def evaluate_acc(model):
    train_score = model.evaluate(x_train, y_train, verbose=0)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])
    val_score = model.evaluate(x_val, y_val, verbose=0)
    print('Validation loss:', val_score[0])
    print('Validation accuracy:', val_score[1])

170500096/170498071 [==============================] - 38s 0us/step
x_train shape: (1000, 32, 32, 3)
x_val shape: (500, 32, 32, 3)
1000 train samples
500 validation samples


### No regularization
First, lets just try seeing how the model performs with no regularization. We are going to train for 150 epochs on 

This should take around 3-5 minutes to train. Go take a break and grab a snack!

In [4]:
model = create_model()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True)
evaluate_acc(model)

W0709 16:16:14.875036 140737189909440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 16:16:14.904582 140737189909440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 16:16:14.909239 140737189909440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0709 16:16:14.955389 140737189909440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0709 16:16:15.046213 140737189909440 deprecation_wrapp

Train on 1000 samples, validate on 500 samples
Epoch 1/150
1000/1000 [==============================] - 4s 4ms/step - loss: 2.2860 - acc: 0.1350 - val_loss: 2.2650 - val_acc: 0.1580
Epoch 2/150
1000/1000 [==============================] - 3s 3ms/step - loss: 2.2381 - acc: 0.2010 - val_loss: 2.2344 - val_acc: 0.2120
Epoch 3/150
1000/1000 [==============================] - 3s 3ms/step - loss: 2.1784 - acc: 0.2440 - val_loss: 2.1664 - val_acc: 0.1740
Epoch 4/150
1000/1000 [==============================] - 3s 3ms/step - loss: 2.0983 - acc: 0.2910 - val_loss: 2.0842 - val_acc: 0.2740
Epoch 5/150
1000/1000 [==============================] - 3s 3ms/step - loss: 2.0171 - acc: 0.3130 - val_loss: 2.0117 - val_acc: 0.3240
Epoch 6/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.9350 - acc: 0.3600 - val_loss: 1.9826 - val_acc: 0.3120
Epoch 7/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8919 - acc: 0.3600 - val_loss: 1.9656 - val_acc: 0.2800
Epoch 8/

Epoch 61/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1908 - acc: 0.5930 - val_loss: 1.7871 - val_acc: 0.3640
Epoch 62/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0984 - acc: 0.6550 - val_loss: 1.7814 - val_acc: 0.3820
Epoch 63/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1506 - acc: 0.6380 - val_loss: 1.8257 - val_acc: 0.3660
Epoch 64/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0845 - acc: 0.6420 - val_loss: 1.7542 - val_acc: 0.3940
Epoch 65/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1420 - acc: 0.6170 - val_loss: 1.7570 - val_acc: 0.3940
Epoch 66/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0864 - acc: 0.6380 - val_loss: 1.7919 - val_acc: 0.3920
Epoch 67/150
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0751 - acc: 0.6500 - val_loss: 1.7548 - val_acc: 0.3900
Epoch 68/150
1000/1000 [========================

1000/1000 [==============================] - 3s 3ms/step - loss: 0.5322 - acc: 0.8590 - val_loss: 2.0858 - val_acc: 0.3740
Epoch 122/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5377 - acc: 0.8460 - val_loss: 2.0622 - val_acc: 0.3900
Epoch 123/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4834 - acc: 0.8710 - val_loss: 2.0723 - val_acc: 0.4060
Epoch 124/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5202 - acc: 0.8550 - val_loss: 2.0311 - val_acc: 0.3860
Epoch 125/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4884 - acc: 0.8620 - val_loss: 2.1576 - val_acc: 0.3840
Epoch 126/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4478 - acc: 0.8910 - val_loss: 2.1710 - val_acc: 0.3580
Epoch 127/150
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5036 - acc: 0.8580 - val_loss: 2.0646 - val_acc: 0.3860
Epoch 128/150
1000/1000 [==============================

### Early Stopping
You should get training accuracy around 0.783, and validation accuracy around 0.37. This is a pretty big gap, one which we cannot improve with just training. Let's first try to add early stopping!

Hints: 
Check out how EarlyStopping is implemented in Keras and how you would go about adding it to your model here: https://keras.io/callbacks/.
You may actually get worse performance initially by adding EarlyStopping. Try tuning the `patience` argument (read up on it in the keras docs). However, while it makes training faster, you likely won't see much improvement in validation accuracy (I get around 2% increase in validation accuracy). Note that while training accuracy may be lower, your validation accuracy will be higher. This is because the model is not super overfit to the training data yet, and has learned some understandable features.

In [5]:

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10)
model = create_model()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

IndentationError: unexpected indent (<ipython-input-5-fd4d0c63ace5>, line 2)

### Weight Decay

Early stopping has helped us not train longer than we need to, but in this case doesn't help much with validation accuracy. This means that the gap between validation and training accuracy is always large, and there are never any "peaky" moments where the distance is decreased. Let's next try to add L2 Weight Decay to our model.

Hint: As example code, you can use the following code to declare a Dense layer with 64 hidden neurons and l2 regularization with weight decay strength 0.01. You can add this to *all* of the *learnable* layers in the model in `create_model`. Check out the documentation on regularizers here: https://keras.io/regularizers/

```
from keras import regularizers

Dense(64, input_dim=64, kernel_regularizer=regularizers.l2(0.01))
```

Add it now to the model in `create_model`, and retrain it here to see your new results. Note that you can tweak your weight decay strength as a hyperparameter. However, I have personally done this and found best results with `0.01`. Can you see if you can outperform this? I personally see about a 1% improvement, and a much smaller gap between training accuracy and validation accuracy. This is due to the fact that while we are less overfit to our training data, we still learning valuable features of the images.

In [ ]:
def create_model_weight_decay():
    """ Creates a new convolutional neural network with weight decay and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    reg = regularizers.l2(0.01)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=reg))
    model.add(Activation('relu'))
    model.add(Dense(num_classes, kernel_regularizer=reg))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
early_stopping = EarlyStopping(patience=10)
model = create_model_weight_decay()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

### Dropout
We next add dropout layers to our model.

In [ ]:
def create_model_dropout():
    """ Creates a new convolutional neural network with weight decay and returns it """
    seed(1)
    input_shape = (32, 32, 3)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
early_stopping = EarlyStopping(patience=10)
model = create_model_dropout()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[early_stopping])
evaluate_acc(model)

I get around 41% validation accuracy using dropout.